In [1]:
import pandas as pd 
import numpy as np
from requests import get
import requests
from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time

In [2]:
import re

In [3]:
#def buscadeveiculosolx (pages, regiao):
driver = webdriver.Chrome(executable_path='./chr_drv/chromedriver.exe')
#soup = BeautifulSoup(driver.page_source, 'html.parser')
url = "https://lista.mercadolivre.com.br/veiculos/carros-caminhonetes/#deal_print_id=9f973e90-e73e-11eb-b5cc-d5f53f9f352d&c_id=carousel&c_element_order=1&c_campaign=CARROS-E-CAMINHONETES&c_uid=9f973e90-e73e-11eb-b5cc-d5f53f9f352d"
pagina = driver.get(url)


#driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/div/div[2]/div[2]/div/div[5]/div/div/div/form/div[2]/div/button").click()
    
  

In [4]:
time.sleep(3)
valor = "45000"
caixadevalor = driver.find_element_by_css_selector('[data-testid="Maximum-price"]')
caixadevalor.clear()
caixadevalor.send_keys(valor)

In [5]:
driver.find_element_by_css_selector('[data-testid="submit-price"]').click()
time.sleep(2)
driver.find_element_by_id("newCookieDisclaimerButton").click()

In [6]:
carro = []
vale = []
links = []
count=1
n =3
v=n
if n>5:
    v=5

In [7]:
#driver.find_elements_by_css_selecto



for j in range(n):
    time.sleep(v)
    
    carros= driver.find_elements_by_xpath('/html/body/main/div/div/section/ol[*]/li[*]/div/div/a/div/div[*]/h2')
   
    pula_pag = driver.find_element_by_css_selector('[title="Seguinte"]')
    preço= driver.find_elements_by_xpath("/html/body/main/div/div/section/ol[*]/li[*]/div/div/a/div/div[*]/div/div/span/span[2]/span[2]")
    
    list_links = driver.find_elements_by_xpath('/html/body/main/div/div/section/ol[*]/li[*]/div/div/a')

    for i in list_links:
        links.append( i.get_attribute('href'))
    for k in carros:
        carro.append(k.text)
            
    for q in preço:
       
        vale.append(q.text)
    
        
    if count<n:
        count+=1
        pula_pag.click()
       


In [8]:
carr = pd.DataFrame(carro)
price = pd.DataFrame(vale).astype("float")
url =pd.DataFrame(links)



In [9]:
pd.get_option('max_colwidth')

50

In [10]:
pd.set_option('max_colwidth',None)

In [11]:
carr.rename({0:"Nome"}, axis=1,inplace=True)
price.rename({0:"Valor em milhar"},axis=1, inplace=True)
url.rename({0:"URL"}, axis=1, inplace = True)


In [12]:
data = pd.concat([carr,price,url],axis=1)

In [17]:
data2 = data.sort_values('Valor em milhar',ascending=True )
data2.reset_index(drop=True, inplace=True)

In [14]:
data2.to_csv('Carros.csv',sep=',', encoding = 'utf-8-sig')

In [15]:
data2.loc[(data2.Nome=="Renault Clio Clio Hatch. Expression 1.0 16v (flex)")]

,Nome,Valor em milhar,URL
